#### 교차 검증 (CV, Cross Validation)
- 기존 방식에서는 데이터 세트에서 학습 데이터 세트와 테스트 데이터 세트를 분리한 뒤, 모델 검증을 진행한다.
- 교차 검증 시, 학습 데이터를 다시 분할하여 학습 데이터와 모델 성능을 1차 평가하는 검증 데이터로 나눈다.

<img src="./images/cross_validation01.png" width="500px">

#### 교차 검증의 장단점
- 👍특정 데이터 세트에 대한 과적합 방지
- 👍데이터 세트 규모가 적을 시 과소적합 방지
- 👎모델 훈련, 모델 평가에 소요되는 시간 증가
- 즉, 과적합을 피하고 하이퍼 파라미터를 튜닝함으로써 모델을 일반화하고 신뢰성을 증가시키기 위한 목적이다.

#### 교차 검증의 종류

K-Fold
- k개의 데이터 폴드 세트를 만든 뒤 k번만큼 학습과 검증 평가를 반복하여 수행하는 방식
- 학습 데이터와 검증 데이터를 정확히 자르기 때문에 타겟 데이터의 비중이 한 곳으로 치중될 수 있다.
- 예를 들어, 0, 1, 2 중에서 0, 1의 두 가지만 잘라서 검증하게 되면 다른 하나의 타겟 데이터(2)를 예측할 수 없게 된다.
- 이 경우, Stratified K-Fold를 사용함으로서 해결한다.

Stratified K-Fold
- K-Fold와 마찬가지로 k번 수행하지만, 폴드 세트를 만들 때 학습 데이터 세트와 검증 데이터 세트가 가지는 타켓의 분포도가 유사하도록 검증한다.
- 타겟 데이터의 비중을 항상 똑같게 자르기 때문에 데이터가 한 곳으로 치중되는 것을 방지한다.

<img src="./images/cross_validation02.png" width="600px">

GridSearchCV
- 교차 검증과 최적의 하이퍼 파라미터 튜닝을 한 번에 할 수 있는 객체이다.
- max_depth와 min_samples_split에 1차원 정수형 list를 전달하면, 2차원으로 결합하여 격자(Grid)를 만들고, 이 중 최적의 점을 찾아낸다.
- 딥러닝에서는 학습 속도가 머신러닝에 비해 느리고, 레이어(층)가 깊어질 수록 조정해주어야 할 하이퍼 파라미터 값이 많아지기 때문에, RandomSearchCV에서 대략적인 범위를 찾은 다음, GridSearchCV로 디테일을 조정하는 방식을 사용한다.

<img src="./images/grid_search_cv.png" width="500px">

In [1]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

iris = load_iris()
features = iris.data
targets = iris.target

target_df = pd.DataFrame(targets, columns=['target'])
target_df.value_counts()

target
0         50
1         50
2         50
Name: count, dtype: int64

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

dtc = DecisionTreeClassifier(random_state=124, min_samples_leaf=6)
kfold = KFold(n_splits=5)

In [3]:
features.shape

(150, 4)

In [7]:
count = 0

for train_index, test_index in kfold.split(features):
    # 데이터 세트 분리
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = targets[train_index], targets[test_index]

    # 학습 및 예측
    dtc.fit(X_train, y_train)
    prediction = dtc.predict(X_test)

    # 평가
    accuracy = np.round(accuracy_score(y_test, prediction), 4)

    # 검증
    train_targets = pd.DataFrame(y_train)
    test_targets = pd.DataFrame(y_test)

    count += 1

    print(f'{count}회차')
    print(f'학습 타겟 데이터 분포: \n{train_targets.value_counts()}')
    print(f'검증 타겟 데이터 분포: \n{test_targets.value_counts()}')

    print(f'정확도: {accuracy}')

1회차
학습 타겟 데이터 분포: 
1    50
2    50
0    20
Name: count, dtype: int64
검증 타겟 데이터 분포: 
0    30
Name: count, dtype: int64
정확도: 1.0
2회차
학습 타겟 데이터 분포: 
2    50
1    40
0    30
Name: count, dtype: int64
검증 타겟 데이터 분포: 
0    20
1    10
Name: count, dtype: int64
정확도: 1.0
3회차
학습 타겟 데이터 분포: 
0    50
2    50
1    20
Name: count, dtype: int64
검증 타겟 데이터 분포: 
1    30
Name: count, dtype: int64
정확도: 0.8333
4회차
학습 타겟 데이터 분포: 
0    50
1    40
2    30
Name: count, dtype: int64
검증 타겟 데이터 분포: 
2    20
1    10
Name: count, dtype: int64
정확도: 0.9333
5회차
학습 타겟 데이터 분포: 
0    50
1    50
2    20
Name: count, dtype: int64
검증 타겟 데이터 분포: 
2    30
Name: count, dtype: int64
정확도: 0.8333


In [8]:
from sklearn.model_selection import StratifiedKFold

s_kfold = StratifiedKFold(n_splits=5)

In [11]:
count = 0
accuracy_list = []

for train_index, test_index in s_kfold.split(features, targets):
    # 데이터 세트 분리
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = targets[train_index], targets[test_index]

    # 학습 및 예측
    dtc.fit(X_train, y_train)
    prediction = dtc.predict(X_test)

    # 평가
    accuracy = np.round(accuracy_score(y_test, prediction), 4)
    accuracy_list.append(accuracy)

    # 검증
    train_targets = pd.DataFrame(y_train)
    test_targets = pd.DataFrame(y_test)

    count += 1

    print(f'{count}회차')
    print(f'학습 타겟 데이터 분포: \n{train_targets.value_counts()}')
    print(f'검증 타겟 데이터 분포: \n{test_targets.value_counts()}')

    print(f'정확도: {accuracy}')

print(f'평균 정확도: {np.mean(accuracy_list)}')

1회차
학습 타겟 데이터 분포: 
0    40
1    40
2    40
Name: count, dtype: int64
검증 타겟 데이터 분포: 
0    10
1    10
2    10
Name: count, dtype: int64
정확도: 0.9667
2회차
학습 타겟 데이터 분포: 
0    40
1    40
2    40
Name: count, dtype: int64
검증 타겟 데이터 분포: 
0    10
1    10
2    10
Name: count, dtype: int64
정확도: 0.9667
3회차
학습 타겟 데이터 분포: 
0    40
1    40
2    40
Name: count, dtype: int64
검증 타겟 데이터 분포: 
0    10
1    10
2    10
Name: count, dtype: int64
정확도: 0.9
4회차
학습 타겟 데이터 분포: 
0    40
1    40
2    40
Name: count, dtype: int64
검증 타겟 데이터 분포: 
0    10
1    10
2    10
Name: count, dtype: int64
정확도: 0.8667
5회차
학습 타겟 데이터 분포: 
0    40
1    40
2    40
Name: count, dtype: int64
검증 타겟 데이터 분포: 
0    10
1    10
2    10
Name: count, dtype: int64
정확도: 1.0
평균 정확도: 0.94002


#### 편하게 수행할 수 있는 교차 검증

**cross_val_score(estimator, x, y, cv, scoring)**

- estimator: classifier 종류 모델이면 내부적으로 stratified K-Fold로 진행된다.
- x: features
- y: targets
- cv: 폴드 세트 개수
- scoring: 평가 함수, 정확도(accuracy)외에 다른 것은 다른 챕터에서 배운다.

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
dtc = DecisionTreeClassifier(random_state=124, min_samples_leaf=6)

features = iris.data
targets = iris.target

score = cross_val_score(dtc, features, targets, cv=5, scoring='accuracy')
print(np.round(np.mean(score), 4))

0.94


#### GridSearchCV
**GridSearchCV(estimator, param_grid, cv, refit, return_train_score)**

- estimator: 학습할 모델 객체 작성
- param_grid: dict 형태로 전달해야 하며, 주요 key 값은 max_depth, min_samples_split이다.
- cv: 폴드 세트 개수
- refit: 전달한 모델 객체에 최적의 하이퍼 파라미터로 적용하고자 할 때 True를 전달한다.
- return_train_score: 교차 검증 점수를 가져올 지에 대해 True 또는 False를 전달한다.

In [16]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV, train_test_split

iris = load_iris()

features = iris.data
targets = iris.target

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=124)

dtc = DecisionTreeClassifier()
parameters = {'max_depth': [2, 3, 4], 'min_samples_split': [6, 7]}

In [17]:
g_dtc = GridSearchCV(dtc,
                     param_grid=parameters,
                     cv=5,
                     refit=True,
                     return_train_score=True,
                     n_jobs=-1)

In [18]:
g_dtc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4], 'min_samples_split': [6, 7]},
             return_train_score=True)

In [19]:
g_dtc.cv_results_

{'mean_fit_time': array([0.        , 0.00080652, 0.00199194, 0.00113726, 0.00012293,
        0.00150294]),
 'std_fit_time': array([0.        , 0.00104282, 0.00303254, 0.00227451, 0.00024586,
        0.00232935]),
 'mean_score_time': array([0.        , 0.00152259, 0.00186248, 0.00214019, 0.00159178,
        0.00131216]),
 'std_score_time': array([0.        , 0.00202831, 0.0020734 , 0.00243606, 0.00318356,
        0.0023606 ]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[6, 7, 6, 7, 6, 7],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split': 6},
  {'max_depth': 2, 'min_samples_split': 7},
  {'max_depth': 3, 'min_samples_split': 6},
  {'max_depth': 3, 'min_samples_split': 7},
  {'max_depth': 4, 'min_sample

In [20]:
result_df = pd.DataFrame(g_dtc.cv_results_)
result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000000,0.000000,0.000000,0.000000,2,6,"{'max_depth': 2, 'min_samples_split': 6}",1.0,0.958333,0.958333,1.0,0.958333,0.975000,0.020412,5,0.96875,0.979167,0.979167,0.96875,0.979167,0.975,0.005103
1,0.000807,0.001043,0.001523,0.002028,2,7,"{'max_depth': 2, 'min_samples_split': 7}",1.0,0.958333,0.958333,1.0,0.958333,0.975000,0.020412,5,0.96875,0.979167,0.979167,0.96875,0.979167,0.975,0.005103
2,0.001992,0.003033,0.001862,0.002073,3,6,"{'max_depth': 3, 'min_samples_split': 6}",1.0,1.000000,0.958333,1.0,1.000000,0.991667,0.016667,1,1.00000,1.000000,1.000000,1.00000,1.000000,1.000,0.000000
3,0.001137,0.002275,0.002140,0.002436,3,7,"{'max_depth': 3, 'min_samples_split': 7}",1.0,1.000000,0.958333,1.0,1.000000,0.991667,0.016667,1,1.00000,1.000000,1.000000,1.00000,1.000000,1.000,0.000000
4,0.000123,0.000246,0.001592,0.003184,4,6,"{'max_depth': 4, 'min_samples_split': 6}",1.0,1.000000,0.958333,1.0,1.000000,0.991667,0.016667,1,1.00000,1.000000,1.000000,1.00000,1.000000,1.000,0.000000
5,0.001503,0.002329,0.001312,0.002361,4,7,"{'max_depth': 4, 'min_samples_split': 7}",1.0,1.000000,0.958333,1.0,1.000000,0.991667,0.016667,1,1.00000,1.000000,1.000000,1.00000,1.000000,1.000,0.000000


In [22]:
print(g_dtc.best_params_, g_dtc.best_score_, sep="\n")

{'max_depth': 3, 'min_samples_split': 6}
0.9916666666666668


In [23]:
g_dtc.best_estimator_

DecisionTreeClassifier(max_depth=3, min_samples_split=6)

In [24]:
dtc = g_dtc.best_estimator_

prediction = dtc.predict(X_test)
accuracy_score(y_test, prediction)

0.9